# Macro data download using FredApi.

In [13]:
from fredapi import Fred
import pandas as pd
import pandas_datareader as pdr

def download_data(tickers, api_key):
    """
    Download data from FRED API.

    Parameters:
    - tickers (list): Tickers list.
    - api_key (str): API Key.

    Returns:
    - DataFrame: DataFrame with data.
    """
    
    # Initialize object with API key
    fred = Fred(api_key=api_key)

    # Create a new empty DataFrame
    df = pd.DataFrame()

    # Download data for every ticker and add them to the DataFrame
    for ticker in tickers:
        data = fred.get_series(ticker)
        df[ticker] = data

    # Set DATE as index name
    df.index.name = 'DATE'
    
    # Convert index to datetime
    df.index = pd.to_datetime(df.index)
    
    print('Download complete')
    
    return df


api_key = 'ebfe58ad7a8159a6f37d81497c8c185c'
tickers = ['GS2', 'GS10', 'CPIAUCSL', 'PPIACO', 'CPILFESL', 'EXUSEU']

df = download_data(tickers, api_key)

Download complete


In [17]:
# Clean from NaNs data
df_clean = df.dropna()

# Print df
df_clean

,GS2,GS10,CPIAUCSL,PPIACO,CPILFESL,EXUSEU
DATE,,,,,,
1999-01-01,4.62,4.72,164.700,122.900,175.600,1.1591
1999-02-01,4.88,5.00,164.700,122.300,175.600,1.1203
1999-03-01,5.05,5.23,164.800,122.600,175.700,1.0886
1999-04-01,4.98,5.18,165.900,123.600,176.300,1.0701
1999-05-01,5.25,5.54,166.000,124.700,176.500,1.0630
...,...,...,...,...,...,...
2023-08-01,4.90,4.17,306.269,257.680,309.661,1.0910
2023-09-01,5.02,4.38,307.481,258.859,310.661,1.0672
2023-10-01,5.07,4.80,307.619,255.042,311.365,1.0565


# Fama French data download using Panda DataReader.

In [18]:
# Defining data set from pandas datareader
data_set = 'F-F_Research_Data_Factors'

# Downloading data
data = pdr.famafrench.FamaFrenchReader(data_set, start='1996-01-01', end='2024-01-01').read()

C:\Users\Sergi Garcia Marsol\AppData\Local\Temp\ipykernel_18820\2481304438.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pdr.famafrench.FamaFrenchReader(data_set, start='1996-01-01', end='2024-01-01').read()
C:\Users\Sergi Garcia Marsol\AppData\Local\Temp\ipykernel_18820\2481304438.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pdr.famafrench.FamaFrenchReader(data_set, start='1996-01-01', end='2024-01-01').read()


In [19]:
# Selecting data
Fama_French_data = pd.DataFrame(data[0])

# Naming index
Fama_French_data.index.name = 'DATE'
# Converting index to_timestamp
Fama_French_data.index = Fama_French_data.index.to_timestamp()

# Cleaning from NaNs data
Fama_French_data = Fama_French_data.dropna()

# Print dataframe
Fama_French_data

,Mkt-RF,SMB,HML,RF
DATE,,,,
1996-01-01,2.26,-2.73,0.38,0.43
1996-02-01,1.33,1.79,-1.08,0.39
1996-03-01,0.73,1.49,0.35,0.39
1996-04-01,2.06,5.23,-4.02,0.46
1996-05-01,2.36,3.17,-0.83,0.42
...,...,...,...,...
2023-08-01,-2.39,-3.16,-1.06,0.45
2023-09-01,-5.24,-2.51,1.52,0.43
2023-10-01,-3.19,-3.87,0.19,0.47


# Merging Fama French and Macro Data

In [22]:
# Merging Fama French and Macro data
macro_df = Fama_French_data.merge(df_clean, how='inner', left_index=True, right_index=True)

# Print dataframe
macro_df

,Mkt-RF,SMB,HML,RF,GS2,GS10,CPIAUCSL,PPIACO,CPILFESL,EXUSEU
DATE,,,,,,,,,,
1999-01-01,3.50,0.75,-4.60,0.35,4.62,4.72,164.700,122.900,175.600,1.1591
1999-02-01,-4.08,-6.08,1.92,0.35,4.88,5.00,164.700,122.300,175.600,1.1203
1999-03-01,3.45,-3.80,-2.74,0.43,5.05,5.23,164.800,122.600,175.700,1.0886
1999-04-01,4.33,3.91,2.46,0.37,4.98,5.18,165.900,123.600,176.300,1.0701
1999-05-01,-2.46,3.34,2.35,0.34,5.25,5.54,166.000,124.700,176.500,1.0630
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-2.39,-3.16,-1.06,0.45,4.90,4.17,306.269,257.680,309.661,1.0910
2023-09-01,-5.24,-2.51,1.52,0.43,5.02,4.38,307.481,258.859,310.661,1.0672
2023-10-01,-3.19,-3.87,0.19,0.47,5.07,4.80,307.619,255.042,311.365,1.0565


In [ ]:
# Saving as excel file
macro_df.to_excel('macro_df.xlsx', index=True)